In [17]:
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 0)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value

In [89]:
def split_in_lines(gray_img, bin_img, min_line_height = 10):
    '''
    function that split an image into lines
    '''
    
    # dilate bin image
#     mask = disk(2)
    bin_img_dilated = dilation(bin_img)

    # count
    ones = np.sum(bin_img_dilated,1)

    # histogram
    median = np.median(ones * 1.0) / 5
    histo = (ones > median) * 1

    # get rising and falling edges from histogram
    shifted = np.roll(histo, -1, 0)
    shifted[-1] = histo[-1]
    edges = histo - shifted
    rising_indices = np.array((edges == -1).nonzero()).flatten()
    falling_indices = np.array((edges == 1).nonzero()).flatten()
    if len(falling_indices) < 2 or len(rising_indices) < 2:
        return [gray_img], [bin_img]
    # make starting with rising not falling 
    if falling_indices[0] < rising_indices[0]:
        falling_indices = falling_indices[1:]
    # make ending with falling not rising 
    if rising_indices[-1] > falling_indices[-1]:
        rising_indices = rising_indices[:-1]

    # cut image on histo
    gray_lines = []
    bin_lines = []
    line_count = min(rising_indices.shape[0], falling_indices.shape[0])
    for i in range(line_count):
        line_height = falling_indices[i] - rising_indices[i]
        # 1/4 of the line as padding
        start_split = max(rising_indices[i] - line_height//4, 0)
        end_split = min(falling_indices[i] + line_height//4, gray_img.shape[0])
        # split with padding
        gray_line = gray_img[start_split:end_split]
        bin_line = bin_img[start_split:end_split]
#         # split binary 
#         bin_line = bin_img[rising_indices[i]:falling_indices[i]]
#         # pad binary
#         bin_line = np.pad(bin_line, line_height//4, pad_with)
        # filter if less than 10 pixels
        if line_height > min_line_height:
            gray_lines.append(gray_line)
            bin_lines.append(bin_line)
    return gray_lines, bin_lines
    

In [90]:
from skimage import io
import os
import torch
import numpy as np
from skimage.color import rgb2gray
import time
from skimage.morphology import disk, dilation, erosion
import os, shutil


paths = os.listdir('./samples')

for p in paths:
# p = '1.jpeg'
    img =  rgb2gray(io.imread('./samples/' + p))
    # clip
    gray_img = img[int(0.4*img.shape[0]):int(0.8*img.shape[0]), int(0.1*img.shape[1]):int(0.9*img.shape[1])]

    # binary
    bin_img = 1 - ((gray_img > 0.7) * 1)   

    start_time = time.time()
    gray_lines, bin_lines = split_in_lines(gray_img,bin_img)
    print("--- %s seconds taken ---" % (time.time() - start_time))
    
    # prepare empty folders
    path = 'lines/' + p.split('.')[0]
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        for filename in os.listdir(path):
            os.unlink(path + '/' + filename)
    
    # save lines
    for i in range(len(gray_lines)):
        io.imsave(path + "/gray_line " + str(i) + '_ sample ' + p + ".png", gray_lines[i])
        io.imsave(path + "/bin_line " + str(i) + '_ sample ' + p + ".png", bin_lines[i])
#         io.imsave("lines/gray_line" + str(i) + ".png", gray_lines[i])
#         io.imsave("lines/bin_line" + str(i) + ".png", bin_lines[i])


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/6/bin_line 0_ sample 6.jpeg.png is a low contrast image
  io.imsave(path + "/bin_line " + str(i) + '_ sample ' + p + ".png", bin_lines[i])
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/6/bin_line 1_ sample 6.jpeg.png is a low contrast image
  io.imsave(path + "/bin_line " + str(i) + '_ sample ' + p + ".png", bin_lines[i])
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-90-ea8

--- 0.02689957618713379 seconds taken ---


<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/6/bin_line 3_ sample 6.jpeg.png is a low contrast image
  io.imsave(path + "/bin_line " + str(i) + '_ sample ' + p + ".png", bin_lines[i])
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/6/bin_line 4_ sample 6.jpeg.png is a low contrast image
  io.imsave(path + "/bin_line " + str(i) + '_ sample ' + p + ".png", bin_lines[i])
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/6/bin_line 5_ sample 6.jpeg.png is a low contrast image
  io.imsave(path + "/bin_line

--- 0.0356144905090332 seconds taken ---


<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/2/bin_line 2_ sample 2.jpeg.png is a low contrast image
  io.imsave(path + "/bin_line " + str(i) + '_ sample ' + p + ".png", bin_lines[i])
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/2/bin_line 3_ sample 2.jpeg.png is a low contrast image
  io.imsave(path + "/bin_line " + str(i) + '_ sample ' + p + ".png", bin_lines[i])
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/2/bin_line 4_ sample 2.jpeg.png is a low contrast image
  io.imsave(path + "/bin_line

...

--- 0.029869556427001953 seconds taken ---


<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/8/bin_line 0_ sample 8.png.png is a low contrast image
  io.imsave(path + "/bin_line " + str(i) + '_ sample ' + p + ".png", bin_lines[i])
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/1/bin_line 0_ sample 1.jpeg.png is a low contrast image
  io.imsave(path + "/bin_line " + str(i) + '_ sample ' + p + ".png", bin_lines[i])
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/1/bin_line 1_ sample 1.jpeg.png is a low contrast image
  io.imsave(path + "/bin_line " + str(i) + '_ sample ' + p + ".png", bin_lines[i])
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

--- 0.027120351791381836 seconds taken ---


<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/1/bin_line 3_ sample 1.jpeg.png is a low contrast image
  io.imsave(path + "/bin_line " + str(i) + '_ sample ' + p + ".png", bin_lines[i])
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/1/bin_line 4_ sample 1.jpeg.png is a low contrast image
  io.imsave(path + "/bin_line " + str(i) + '_ sample ' + p + ".png", bin_lines[i])
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-90-ea81ed95de32>:37: UserWarning: lines/1/bin_line 5_ sample 1.jpeg.png is a low contrast image
  io.imsave(path + "/bin_line

--- 0.02929067611694336 seconds taken ---


In [ ]:
# torch
def split_in_lines(img, min_line_height = 10):
    '''
    function that split an image into lines
    '''
    # clip
    img = img[int(0.25*img.shape[0]):int(0.7*img.shape[0]), int(0.1*img.shape[1]):int(0.9*img.shape[1])]

    # binary
    img = 1 - ((img > 128) * 1)

    # torch tensor
    img_tensor = torch.tensor(img)

    # count
    ones = torch.sum(img_tensor,1)

    # histogram
    mean = torch.mean(ones * 1.0) / 5
    histo = (ones > mean) * 1

    # get rising and falling edges from histogram
    shifted = torch.roll(histo, -1, 0)
    shifted[-1] = histo[-1]
    edges = histo - shifted
    rising_indices = torch.flatten((edges == -1).nonzero())
    falling_indices = torch.flatten((edges == 1).nonzero())
    if len(falling_indices) < 2 or len(rising_indices) < 2:
        return img
    # make starting with rising not falling 
    if falling_indices[0] < rising_indices[0]:
        falling_indices = falling_indices[1:]
    # make ending with falling not rising 
    if rising_indices[-1] > falling_indices[-1]:
        rising_indices = rising_indices[:-1]

    # cut image on histo
    lines = []
    line_count = min(rising_indices.size()[0], falling_indices.size()[0])
    for i in range(line_count):
        line = img[rising_indices[i]:falling_indices[i]]
        line_height = line.shape[0]
        line = np.pad(line, line.shape[0]//3, pad_with)
        # filter if less than 10 pixels
        if line_height > min_line_height:
            lines.append(1-line)
    return lines
    

In [ ]:
from skimage import io
import os
import torch
import numpy as np
paths = os.listdir('../data/formsA-D')

for p in paths:
    img = io.imread('../data/formsA-D/' + p)
    lines = split_in_lines(img)

    for i in range(len(lines)):
        io.imsave("lines/line" + str(i) + '_' + p + ".png", lines[i])